In [77]:
# %pip install dotenv requests oauthlib requests_oauthlib

In [78]:
import os
from dotenv import load_dotenv

load_dotenv()
OAUTH2_CLIENT_ID = os.environ["OAUTH2_CLIENT_ID"]
OAUTH2_CLIENT_SECRET = os.environ["OAUTH2_CLIENT_SECRET"]
OAUTH2_TOKEN_ENDPOINT = os.environ["OAUTH2_TOKEN_ENDPOINT"]
COPERNICUS_WMS_ENDPOINT = os.environ["COPERNICUS_WMS_ENDPOINT"]

os.chdir("/home/me/workspace/det_remota/trabalho_final")

In [79]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Create a session
client = BackendApplicationClient(client_id=OAUTH2_CLIENT_ID)
oauth = OAuth2Session(client=client)

# Get token for the session
OAUTH_TOKEN = ""
def refresh_token(): 
    global OAUTH_TOKEN
    OAUTH_TOKEN = oauth.fetch_token(
        token_url=OAUTH2_TOKEN_ENDPOINT,
        client_secret=OAUTH2_CLIENT_SECRET,
        include_client_id=True
    )
refresh_token()


# All requests using this session will have an access token automatically added
resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
resp.json()

[{'@id': 'https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances/6aab06db-528b-441e-a958-9743a3ed06a3',
  'id': '6aab06db-528b-441e-a958-9743a3ed06a3',
  'name': 'Sentinel-2 L2A',
  'domainAccountId': '17b3c6c5-72c3-4c63-aa70-4ad5dd877c6f',
  'additionalData': {'showWarnings': True,
   'showLogo': True,
   'imageQuality': 90},
  'created': '2025-04-11T23:21:35.358583Z',
  'lastUpdated': '2025-04-11T23:22:17.001865Z',
  'layers': {'@id': 'https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances/6aab06db-528b-441e-a958-9743a3ed06a3/layers'}}]

In [80]:
from typing import Literal
import datetime

GET_MAP = "GetMap"
GET_CAPABILITIES = "GetCapabilities"

def wms_request(request: Literal["GetMap", "GetCapabilities"], params: dict = dict()):

    token_expires_at = datetime.datetime.fromtimestamp(OAUTH_TOKEN['expires_at'])
    cur_time = datetime.datetime.now()
    if token_expires_at < cur_time:
        print(f"Current time is {cur_time} and token expired at {token_expires_at}")
        print("Refreshing token")
        refresh_token()

    return oauth.get(COPERNICUS_WMS_ENDPOINT, params=dict(
        SERVICE="WMS",
        VERSION="1.3.0",
        REQUEST=request,
        **params
    ))

def get_capabilities():
    return wms_request(GET_CAPABILITIES).text

print(get_capabilities())

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<WMS_Capabilities version="1.3.0" xsi:schemaLocation="https:/inspire.ec.europa.eu/schemas/inspire_vs/1.0 https://inspire.ec.europa.eu/schemas/inspire_vs/1.0/inspire_vs.xsd" xmlns:inspire_common="https://inspire.ec.europa.eu/schemas/common/1.0" xmlns:inspire_vs="https://inspire.ec.europa.eu/schemas/inspire_vs/1.0" xmlns="http://www.opengis.net/wms" xmlns:sld="http://www.opengis.net/sld" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">	<Service>
		<Name>WMS</Name>
		<Title>Sentinel Hub WMS service - Sentinel-2 L2A</Title>
		<Abstract>
<![CDATA[The Copernicus project's Sentinel satellites are revolutionizing earth observation (EO). Its free, full and open access to data with very short revisit times, high spatial resolution, and good spectral resolution are crucial for many applications. The portfolio of possible products is vast - use-cases of such a service range from plant health mon

### GetMap Parameters

|WMS parameter | Info|
|---------------|---------|
|BBOX | Specifies the bounding box of the requested image. Coordinates must be in the specified coordinate reference system. The four coordinates representing the top-left and bottom-right of the bounding box must be separated by commas. Required. Example: BBOX=-13152499,4038942,-13115771,4020692|
|CRS | (when VERSION 1.3.0 or higher) the coordinate reference system in which the BBOX is specified and in which to return the image. Optional, default: "EPSG:3857". For a list of available CRSs see the GetCapabilities result.|
|SRS | (when VERSION 1.1.1 or lower) the coordinate reference system in which the BBOX is specified and in which to return the image. Optional, default: "EPSG:3857". For a list of available CRSs see the GetCapabilities result.|
|FORMAT | The returned image format. Optional, default: "image/png", other options: "image/jpeg", "image/tiff". Detailed information about supported values.|
|WIDTH | Returned image width in pixels. Required, unless RESX is used. If WIDTH is used, HEIGHT is also required.|
|HEIGHT | Returned image height in pixels. Required, unless RESY is used. If HEIGHT is used, WIDTH is also required.|
|RESX | Returned horizontal image resolution in UTM units (if m is added, e.g. 10m, in metrical units). (optional instead of WIDTH). If used, RESY is also required.|
|RESY | Returned vertical image resolution in UTM units (if m is added, e.g. 10m, in metrical units). (optional instead of HEIGHT). If used, RESX is also required.|
|LAYERS | The preconfigured layer (image) to be returned. You must specify exactly one layer and optionally add additional overlays. Required. Example: LAYERS=TRUE_COLOR,OUTLINE|
|EXCEPTIONS | The exception format. Optional, default: "XML". Supported values: "XML", "INIMAGE", "BLANK" (all three for version >= 1.3.0), "application/vnd.ogc.se_xml", "application/vnd.ogc.se_inimage", "application/vnd.ogc.se_blank" (all three for version < 1.3.0).|

In [81]:
%conda install pillow

Channels:
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [82]:
from typing import Optional
from io import BytesIO
from PIL import Image


TileFormat = Literal["image/png", "image/jpeg", "image/tiff"]
# Ariquemes bounding box
# MINX	MINY	MAXX	MAXY
# -7081948.111609276	-1151449.6232446095	-6948853.905027688	-1081511.717608428
# Exemplo de uma área com vários espelhos dagua
# -7002111.5,-1090829.0,-6986823.9,-1100888.0


STUDY_AREA_BBOX = "-7002111.5,-1090829.0,-6986823.9,-1100888.0"


def get_tile(
    bbox: str = STUDY_AREA_BBOX,
    from_time:Optional[datetime.date]=None,
    to_time:Optional[datetime.date]=None,
    layer:str="COLOR_INFRARED",
    format:TileFormat="image/tiff", 
    max_cc:float=10,
    crs: str = 'EPSG:3857'
):
    if not from_time or not to_time:
        to_time = datetime.date.today()
        from_time = to_time - datetime.timedelta(days=15)
    
    print(f"Downloading layer {layer} in the period from {from_time} to {to_time}")

    response = wms_request(GET_MAP, params=dict(
        TIME=f"{from_time.isoformat()}/{to_time.isoformat()}",
        LAYERS=layer,
        FORMAT=format,
        MAXCC=max_cc,
        BBOX=bbox,
        CRS=crs,
        RESX="20m",
        RESY="20m",
    ))

    if response.status_code == 200:
        return response.content
    
    raise ValueError(f"Request failed with status {response.status_code}: {response.text}")

example_png = get_tile(max_cc=50, format="image/png")
Image.open(BytesIO(example_png)).show()

In [83]:
from pathlib import Path


SENTINEL2_OUT_PATH = Path("data/sentinel2")


def first_weeks_of_month(year:int, month:int):
    first_date = datetime.date(year=year, month=month, day=1)
    last_date = datetime.date(year=year, month=month, day=14)
    return first_date, last_date

def last_weeks_of_month(year:int, month:int):
    first_date = datetime.date(year=year, month=month, day=15)
    next_month = datetime.date(year=year, month=month, day=28) + datetime.timedelta(days=4)
    last_date = next_month - datetime.timedelta(days=next_month.day)
    return first_date, last_date


def download_images(from_year: int = 2016, to_year: int = 2016):
    """
    COLOR_INFRARED is RGB 8-4-3
    SWIR is RGB 12-11-4
    """
    for year in range(from_year, to_year+1):
        for month in range(1, 12):
            for from_date, to_date in [first_weeks_of_month(year, month), last_weeks_of_month(year, month)]:
                for layer in ['COLOR_INFRARED', 'SWIR']:
                    image_bytes = get_tile(
                        from_time=from_date,
                        to_time=to_date,
                        layer=layer,
                        max_cc=100,
                        format="image/tiff",
                    )
                    filename = f'{layer}_{from_date}_{to_date}.tiff'
                    out_file_path = SENTINEL2_OUT_PATH / filename
                    out_file_path.parent.mkdir(parents=True, exist_ok=True)
                    
                    with open(out_file_path, 'wb') as out_file:
                        out_file_path.write_bytes(image_bytes)


download_images(from_year=2016, to_year=2020)